### Import the packages

In [21]:
import os

# Replace with the path you got from the previous command
libomp_path = "/opt/homebrew/opt/libomp/lib"

# Add libomp to the DYLD_LIBRARY_PATH
os.environ['DYLD_LIBRARY_PATH'] = f"{libomp_path}:" + os.environ.get('DYLD_LIBRARY_PATH', '')
%pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install pandas numpy matplotlib seaborn scikit-learn xgboost lifelines

import os
import pandas as pd
import numpy as np
from copy import deepcopy
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from lifelines import KaplanMeierFitter, CoxTimeVaryingFitter

# Ensure libomp is included in the library path
os.environ['DYLD_LIBRARY_PATH'] = f"{libomp_path}:" + os.environ.get('DYLD_LIBRARY_PATH', '')

sns.set_style('whitegrid')

In [22]:
# define column names for easy indexing
index_names = ['unit_num', 'time_cycles']
setting_names = ['setting1', 'setting2', 'setting3']
sensor_names = ['sensor{}'.format(i) for i in range(1, 22)] 
col_names = index_names + setting_names + sensor_names

# read data
train = pd.read_csv('../data/train_FD001.txt', sep='\s+', header=None, names=col_names)
test = pd.read_csv('../data/test_FD001.txt', sep='\s+', header=None, names=col_names)
y_test = pd.read_csv('../data/RUL_FD001.txt', sep='\s+', header=None, names=['RUL'])

# inspect first few rows
train.head()

FileNotFoundError: [Errno 2] No such file or directory: '../data/train_FD001.txt'

### Exploratory data analysis

In [24]:
del_cols += index_names

X_train = train.drop(del_cols, axis=1)
y_train = X_train.pop('RUL')

# Since the true RUL values for the test set are only provided for the last time cycle of each enginge, 
# the test set is subsetted to represent the same
X_test = test.groupby('unit_num').last().reset_index().drop(del_cols, axis=1)

scaler = StandardScaler()
X_train = pd.DataFrame(data=scaler.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(data=scaler.transform(X_test), columns=X_test.columns)

X_train.shape, X_test.shape

NameError: name 'del_cols' is not defined

### Building baseline models

In [25]:
def get_metrics(y, p, label):
    mae = mean_absolute_error(y, p)
    rmse = mean_squared_error(y, p, squared=False)
    r2 = r2_score(y, p)
    print(f'----- {label} -----\nRMSE: {round(rmse, 2)}\nMAE : {round(mae, 2)}\nR2  : {round(r2, 2)}\n')

In [26]:
def train_and_plot(model, X_train, y_train, X_test, y_test, plot):
    model = deepcopy(model)
    model.fit(X_train, y_train)
    
    y_pred_train = model.predict(X_train)
    get_metrics(y_train, y_pred_train, "Train")

    y_pred_test = model.predict(X_test)
    get_metrics(y_test, y_pred_test, "Test")
    
    if plot:
        plt.figure(figsize=(5, 5))

        sns.regplot(x=y_test.RUL.values, y=y_pred_test, color='red', label="Linear fit")
        sns.scatterplot(x=y_test.RUL.values, y=y_pred_test, edgecolor='k')

        plt.xlabel("Actual RUL")
        plt.ylabel("Predicted RUL")
        plt.title("Actual vs predicted RUL for test set")

        plt.show()
    
    return model

**Linear Regression**

In [27]:
lm = LinearRegression()
lm = train_and_plot(lm, X_train, y_train, X_test, y_test, plot=True)

NameError: name 'X_train' is not defined

In [ ]:
rf = RandomForestRegressor(max_depth=10, random_state=42, n_jobs=15)
rf = train_and_plot(rf, X_train, y_train, X_test, y_test, plot=True)

**One big assumption I am making is that RUL decreases linearly over time. Let's revisit that assumption. We see that, in many sensors, the values are often constant initially and there's sudden rise or fall in the values. This makes sense since the engines only develop a fault over time. The bend in the curve of the signal is the first bit of information provided to us that the engine is degrading and the first time it is reasonable to assume RUL is linearly declining. We can’t really say anything about the RUL before that point because we have no information about the initial wear and tear.**

**Therefore, I can update my assumption to reflect this logic. Instead of having the RUL decline linearly, I define the RUL to start out as a constant and only decline linearly after some time. By doing so we achieve two things:**<br><br>

<b>
    <ol>
        <li>Initially constant RUL correlates better with the initially constant mean sensor signal</li>
        <li>Lower peak values for RUL result in lower spread of our target variable, making it easier to fit a line</li>
    </ol>
</b>

In [ ]:
y_train_clipped = y_train.clip(upper=125)  

In [ ]:
lm = LinearRegression()
lm = train_and_plot(lm, X_train, y_train_clipped, X_test, y_test, plot=True)

In [ ]:
rf = RandomForestRegressor(max_depth=8, random_state=42, n_jobs=15)
rf = train_and_plot(rf, X_train, y_train_clipped, X_test, y_test, plot=True)

**This time both RMSE and R<sup>2</sup> improved significantly. RMSE shows an 42% decrease while R<sup>2</sup> increased by 105%. So we're on the right track. Let's check the feature importance score for the model.**

In [ ]:
!python --version

In [ ]:
X_train.columns

In [ ]:
cols =X_train.columns.tolist()


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

sns.barplot(x=cols, y=rf.feature_importances_, ax=axes[0])
sns.lineplot(x=train["sensor11"], y=y_train_clipped, color='red', ax=axes[1])
sns.scatterplot(x=train["sensor11"], y=y_train_clipped, edgecolor='k', ax=axes[1])

axes[0].tick_params('x', labelrotation=90)
axes[0].set_xlabel("Features")
axes[0].set_ylabel("Relative Importance")
axes[0].set_title("Feature importance plot for the RandomForest model")

axes[1].set_xlabel("Sensor 11")
axes[1].set_ylabel("RUL")
axes[1].set_title("Sensor 11 vs RUL plot")

plt.tight_layout()
plt.show()

**The left plot shows Sensor 11 to be highly important. The right plot shows a strong decreasing trend for Sensor 11 with RUL.**

### Survival Analysis

In [ ]:
train2 = train.copy()
train2['RUL'] = train2['RUL'].clip(upper=125)  

train2.drop(del_cols[:-2], axis=1, inplace=True)
train2 = train2.reset_index(drop=True)

remaining_sensors = list(train2.columns)[2:-1]

**Add an event column**

In [ ]:
train2['breakdown'] = 0

idx_last_record = train2.reset_index().groupby(by='unit_num')['index'].last()  # engines breakdown at the last cycle
train2.loc[idx_last_record, 'breakdown'] = 1

**Indicate the start and stop times of each observation**

In [ ]:
train2['start'] = train2['time_cycles'] - 1 
train2.tail()

**In the train set each engine is run to failure, therefore there aren’t any censored observations. We’ll artificially right-censor our dataset by disregarding any records after 200 time_cycles. This allows us to play around with the data in a bit more realistic setting, with a mix of engines which did and did not have their breakdown yet.**

In [ ]:
cut_off = 200
train_censored = train2[train2.time_cycles <= cut_off].copy()

**KaplanMeier curve**

In [ ]:
data = train_censored[index_names + ['breakdown']].groupby('unit_num').last()

plt.figure(figsize=(12, 5))

survival = KaplanMeierFitter()
survival.fit(data['time_cycles'], data['breakdown'])
survival.plot()

plt.ylabel("Probability of survival")
plt.show()

**100% probability of engine surviving before 128 cycles as expected since none of the engines broke down before that. There's also a ~45% probability of surviving past 200 cycles.**

#### Cox Proportional Hazards models

In [ ]:
# remaining_sensors.remove("sensor8")
# remaining_sensors.remove("sensor15")
# remaining_sensors.remove("sensor3")
# remaining_sensors.remove("sensor21")

In [ ]:
train_cols = index_names + remaining_sensors + ['start', 'breakdown']
predict_cols = ['time_cycles'] + remaining_sensors + ['start', 'breakdown']  # breakdown value will be 0

ctv = CoxTimeVaryingFitter()
_ = ctv.fit(train_censored[train_cols], id_col="unit_num", event_col='breakdown', 
            start_col='start', stop_col='time_cycles', show_progress=True, fit_options={"step_size": 1})

In [ ]:
ctv.print_summary()

plt.figure(figsize=(10, 5))
ctv.plot()

plt.show()

**Sensor 8 and 15 have large p-values**

In [ ]:
df1 = train_censored.groupby("unit_num").last()
df1 = df1[df1['breakdown'] == 0]  # get engines from dataset which are still functioning so we can predict their RUL
df_to_predict = df1[df1['breakdown'] == 0].copy()

predictions = ctv.predict_log_partial_hazard(df_to_predict[predict_cols])
predictions = pd.DataFrame(predictions, columns=["predictions"])

df_last = train.groupby('unit_num').last()
predictions['RUL'] = df_to_predict['RUL'].values
predictions.head(10)

In [ ]:
plt.figure(figsize=(12, 5))

sns.scatterplot(x=predictions['RUL'], y=predictions['predictions'])

xlim = plt.gca().get_xlim()
plt.xlim(xlim[1], xlim[0])
plt.xlabel('RUL')
plt.ylabel('log_partial_hazard')

plt.show()

In [ ]:
from scipy.optimize import curve_fit

def exponential_model(z, a, b):
    return a * np.exp(-b * z)

In [ ]:
ctv2 = CoxTimeVaryingFitter()
ctv2.fit(train2[train_cols], id_col="unit_num", event_col='breakdown', 
        start_col='start', stop_col='time_cycles', show_progress=False)

train2['hazard'] = ctv2.predict_log_partial_hazard(train2)
popt2, pcov2 = curve_fit(exponential_model, train2['hazard'], train2['RUL'])

y_hat = exponential_model(train2['hazard'], *popt2)
get_metrics(train2['RUL'], y_hat, 'Train')

y_pred = ctv2.predict_log_partial_hazard(test.groupby('unit_num').last())
y_hat = exponential_model(y_pred, *popt2)
get_metrics(y_test, y_hat, "Test")